In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from fugashi import Tagger

In [14]:
def preprocessing_text(text):
    tagger = Tagger('-Owakati')
    parsed_text = tagger.parse(text)
    parsed_text = parsed_text.replace("[ MASK ]", "[MASK]")
    return parsed_text

In [15]:
text = "京都大学で自然言語処理を[MASK]する。"

# parse text using fugashi(mecab+unidic-lite)
input_text = preprocessing_text(text)

print(text)
print(input_text)

京都大学で自然言語処理を[MASK]する。
京都 大学 で 自然 言語 処理 を [MASK] する 。


In [16]:
# import japanese language model from huggingface
# tokenizer = AutoTokenizer.from_pretrained('ku-nlp/deberta-v2-base-japanese')
# model = AutoModelForMaskedLM.from_pretrained('ku-nlp/deberta-v2-base-japanese')

In [17]:
# import japanese language model from local file
tokenizer = AutoTokenizer.from_pretrained('C:/code/viya4-implementation-of-japanese-ner-model/asset/model')
model = AutoModelForMaskedLM.from_pretrained('C:/code/viya4-implementation-of-japanese-ner-model/asset/model')

In [18]:
encoding = tokenizer(input_text, return_tensors='pt')
input_ids = encoding.input_ids
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
result = model(encoding.input_ids)

pred_list = []
pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]
for pred_id in pred_ids:
    output_ids = input_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    pred_list.append(tokenizer.decode(output_ids))

In [20]:
print(pred_list)

['[CLS] 京都 大学 で 自然 言語 処理 を 専攻 する 。[SEP]', '[CLS] 京都 大学 で 自然 言語 処理 を 研究 する 。[SEP]', '[CLS] 京都 大学 で 自然 言語 処理 を 勉強 する 。[SEP]', '[CLS] 京都 大学 で 自然 言語 処理 を 履修 する 。[SEP]', '[CLS] 京都 大学 で 自然 言語 処理 を 担当 する 。[SEP]']
